#  Exploratory Data Analysis on IPL Data (2008-2019)

#### check if plots are loaded..if not follow steps in next cell....if they are SCROLL AND ENJOY

#### run all cells..there is chance that plots are not saved..because plotly is browser rendered and make sure u have all libs 

#### library installations (run these here..)

In [1]:
!pip install plotly

In [2]:
!pip install opencage

#### make sure u have "map_token.txt" and "geo.txt" in current directory before running

### ------------------------------------------------------ EDA starts here---------------------------------------------------------

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
matches = pd.read_csv("../input/api-tokens/matches_ipl.csv")

In [5]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [6]:
matches = matches.drop(columns  = ['umpire3'])
matches = matches.dropna().reset_index()

## ------------------------------------global data analysis (2008- 2019)--------------------------------------

In [7]:
fig = px.bar(x = matches['player_of_match'].value_counts().index.tolist()[:20],
             y = matches['player_of_match'].value_counts().values.tolist()[:20], 
             title = "top 20 players with most IPL MVPs (2008-2019)",color =matches['player_of_match'].value_counts().index.tolist()[:20] )
fig.update_xaxes(title = "players")
fig.update_yaxes(title = "NO.OF MVPs")
fig.show()

In [8]:
from opencage.geocoder import OpenCageGeocode
geo = OpenCageGeocode(open('../input/api-tokens/geo.txt').read())

In [9]:
latlong_dict = {}
visited = []
for index in matches['city'].value_counts().index.tolist():
    if index not in visited:
        call = geo.geocode(index)
        latlong_dict[index] = [call[0]['geometry']['lat'],call[0]['geometry']['lng']]

In [10]:
hyd_re = geo.geocode("hyderabad, india")
del latlong_dict['Hyderabad']
latlong_dict['Hyderabad'] = [hyd_re[0]['geometry']['lat'],hyd_re[0]['geometry']['lng']]

In [11]:
matches['lat'] =[latlong_dict[index][0] for index in matches['city']]
matches['lng'] =[latlong_dict[index][1] for index in matches['city']]
matches['city_counts'] = [matches['city'].value_counts().to_dict()[index] for index in matches['city']]

In [12]:
px.set_mapbox_access_token(open('../input/api-tokens/map_token.txt').read())

###  zoom out of the map, if it is not auto scaled

In [13]:
fig = px.scatter_mapbox(lat=matches['lat'], lon=matches['lng'], color=matches['city'],size = matches['city_counts'],
                  color_continuous_scale=px.colors.cyclical.IceFire,size_max=15, zoom=10,title = "IPL venues (2007-2019)",height = 700)
fig.show()

In [14]:
def win_trends(matches):
    team_list = pd.DataFrame(columns = ['year','team','wins'])
    for year in sorted(matches['season'].value_counts().index.tolist()):
        year_list = matches[matches['season'] == year]
        for team in year_list['winner'].value_counts().index.tolist():
            team_list = team_list.append({"year":year,"team":team,"wins":year_list['winner'].value_counts()[team]},ignore_index = True)
    return team_list

In [15]:
win_trends = win_trends(matches)
win_trends = win_trends.replace(to_replace = "Rising Pune Supergiants", value = "Rising Pune Supergiant")

In [16]:
fig = px.line(win_trends,x = "year",y ="wins",color = "team",
              hover_name = "team", height = 500,width =1000 ,range_x=[2007,2020], title ="win Trends of teams (2008 - 2019)")
fig.show()

#### disable and enable each trend by clicking on it in legend

In [17]:
fig = px.bar(x = matches['winner'].value_counts().index,
             y = matches['winner'].value_counts().values, 
             title = "most IPL team wins (2008-2019)",color =matches['winner'].value_counts().index )
fig.update_xaxes(title = "teams")
fig.update_yaxes(title = "NO.OF Wins")
fig.show()

In [18]:
fig = px.bar(x = matches[matches['toss_winner'] == matches['winner']]['winner'].value_counts().index,
             y = matches[matches['toss_winner'] == matches['winner']]['winner'].value_counts().values, 
             title = "most IPL team wins given they won toss (2008-2019)",
             color =matches[matches['toss_winner'] == matches['winner']]['winner'].value_counts().index )
fig.update_xaxes(title = "teams")
fig.update_yaxes(title = "NO.OF Wins")
fig.show()

In [19]:
toss_win_match_lose = matches[matches['toss_winner'] != matches['winner']]

In [20]:
fig = px.bar(x = toss_win_match_lose['toss_winner'].value_counts().index,
             y = toss_win_match_lose['toss_winner'].value_counts().values, 
             title = "most IPL team loss given they won toss (2008-2019)",
             color =toss_win_match_lose['toss_winner'].value_counts().index )
fig.update_xaxes(title = "teams")
fig.update_yaxes(title = "NO.OF Wins")
fig.show()

In [21]:
toss_n_match_winners = matches[matches['toss_winner'] == matches['winner']]

#### hover over plots for values

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows =8 ,cols = 1,
                   specs = [[{"type":"bar","rowspan":2}],
                            [None],
                            [{"type":"bar","rowspan":2}],
                            [None],
                            [{"type":"bar","rowspan":2}],
                            [None],
                            [{"type":"bar","rowspan":2}],
                            [None]])
fig.add_trace(go.Bar(x = toss_n_match_winners[toss_n_match_winners['toss_decision'] == "bat"]['winner'].value_counts().index,
                    y =toss_n_match_winners[toss_n_match_winners['toss_decision'] == "bat"]['winner'].value_counts().values,
                     name = "Wins if toss is won and choose to Bat first (2008-2019)"),row =1 ,col =1 )
fig.add_trace(go.Bar(x = toss_n_match_winners[toss_n_match_winners['toss_decision'] == "field"]['winner'].value_counts().index,
                    y =toss_n_match_winners[toss_n_match_winners['toss_decision'] == "field"]['winner'].value_counts().values,
                     name = "Wins if toss is won and choose to field first (2008-2019)"),row =3 ,col =1, )
fig.add_trace(go.Bar(x = toss_win_match_lose[toss_win_match_lose['toss_decision'] == "bat"]['toss_winner'].value_counts().index,
                    y =toss_win_match_lose[toss_win_match_lose['toss_decision'] == "bat"]['toss_winner'].value_counts().values,
                     name = "loss if toss is won and choose to bat first (2008-2019)"),row =5 ,col =1 )
fig.add_trace(go.Bar(x = toss_win_match_lose[toss_win_match_lose['toss_decision'] == "field"]['toss_winner'].value_counts().index,
                    y =toss_win_match_lose[toss_win_match_lose['toss_decision'] == "field"]['toss_winner'].value_counts().values,
                     name = "loss if toss is won and choose to field first (2008-2019)"),row =7 ,col =1 )
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False,title = "wins")
fig.update_layout(title = "no.of wins for each team based on conditions")
fig.update_xaxes(title = "teams")
fig.show()

In [23]:
fig = px.pie(names = matches['dl_applied'].value_counts().index, values= matches['dl_applied'].value_counts().values,title = "percertage of matches, DL is applied for")
fig.show()

In [24]:
deliveries = pd.read_csv("../input/api-tokens/deliveries_ipl.csv")

## -------------------------------------------season wide analysis (2017) ---------------------------------------

In [25]:
matches_2017 = matches[matches['season'] == 2017]
matches_2017.head()

,index,id,season,city,date,team1,team2,toss_winner,toss_decision,result,...,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,lat,lng,city_counts
0,0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,...,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,17.360589,78.474061,64
1,1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,...,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,18.521428,73.854454,38
2,2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,...,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,22.305199,70.802834,10
3,3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,...,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,22.720362,75.868200,9
4,5,6,2017,Hyderabad,2017-04-09,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,...,Sunrisers Hyderabad,0,9,Rashid Khan,"Rajiv Gandhi International Stadium, Uppal",A Deshmukh,NJ Llong,17.360589,78.474061,64


In [26]:
deliveries_2017 = deliveries[deliveries['match_id'].isin([*range(1,60)])]
deliveries_2017.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [27]:
fig = go.Figure(go.Pie(labels = deliveries['dismissal_kind'].value_counts().index, values =deliveries['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2017",pull = [0,0.2,0,0.2,0,0,0.2]))
fig.show()

In [28]:
fig = px.bar(x = matches_2017['winner'].value_counts().index, y =  matches_2017['winner'].value_counts().values,color =  matches_2017['winner'].value_counts().index,
            title = "top IPL team wins in 2017")
fig.show()

In [29]:
from collections import defaultdict 
def synergy_gen(data_frame):
    current_field = [data_frame['batsman'][0],data_frame['non_striker'][0]]
    countered_by = {}
    synergy_dict = defaultdict(int)
    for bat,non,runs,bowler in zip(data_frame['batsman'],data_frame['non_striker'],data_frame['total_runs'],data_frame['bowler']):
        if bat and non in current_field:
            synergy_dict["-".join(current_field)] = synergy_dict["-".join(current_field)] + runs
        else:
            current_field = [bat,non]
            synergy_dict["-".join(current_field)] = synergy_dict["-".join(current_field)] + runs
            countered_by["-".join(current_field)] = bowler
    return pd.DataFrame(sorted(synergy_dict.items(), key = lambda x : x[1],reverse = True))

In [30]:
synergy_2017 = synergy_gen(deliveries_2017) 

In [31]:
fig = px.bar(y = synergy_2017[0].tolist()[:10],x = synergy_2017[1].tolist()[:10],
                       title ="top 10 synergies of 2017 IPL season", orientation = "h",color = synergy_2017[0].tolist()[:10])
fig.update_yaxes(title = "synergies")
fig.update_xaxes(title = "runs")
fig.show()

In [32]:
matches_2017_top = deliveries_2017[deliveries_2017["batting_team"].isin(matches_2017['winner'].value_counts().index.tolist()[:1])]

In [33]:
matches_2017_top_synergy = synergy_gen(matches_2017_top.reset_index())

### synergy of best team of 2017

In [34]:
fig = px.bar(y = matches_2017_top_synergy[0].tolist()[:10],x = matches_2017_top_synergy[1].tolist()[:10],
                       title =f"top 10 synergies of {matches_2017['winner'].value_counts().index.tolist()[:1]} - 2017 ", orientation = "h",color = matches_2017_top_synergy[0].tolist()[:10])
fig.update_yaxes(title = "synergies")
fig.update_xaxes(title = "runs")
fig.show()

## ---------------------------- season wide batch analytics split -1 (2008 - 2013) ---------------------------

In [35]:
yearlist = sorted(matches['season'].value_counts().index.tolist())

In [36]:
matches_first_split = matches[matches['season'].isin(yearlist[:6])]

In [37]:
matches_first_split.head()

,index,id,season,city,date,team1,team2,toss_winner,toss_decision,result,...,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,lat,lng,city_counts
58,59,60,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,...,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium,Asad Rauf,RE Koertzen,12.979120,77.591300,63
59,60,61,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,...,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",MR Benson,SL Shastri,30.733442,76.779714,46
60,61,62,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,...,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla,Aleem Dar,GA Pratapkumar,28.651718,77.221939,73
61,62,63,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,...,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium,SJ Davis,DJ Harper,19.075990,72.877393,101
62,63,64,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,...,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens,BF Bowden,K Hariharan,22.541418,88.357691,77


In [38]:
deliveries_2008 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2008]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2009 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2009]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2010 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2010]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2011 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2011]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2012 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2012]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2013 = deliveries[deliveries['match_id'].isin(sorted(matches_first_split[matches_first_split['season'] == 2013]['id'].value_counts().index.tolist()))].reset_index()

In [39]:
fig = make_subplots(rows =2 ,cols = 3,
                   specs = [[{"type":"pie"},{"type":"pie"},{"type":"pie"}],
                            [{"type":"pie"},{"type":"pie"},{"type":"pie"}]])
fig.add_trace(go.Pie(labels = deliveries_2008['dismissal_kind'].value_counts().index, values =deliveries_2008['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2008",pull = [0,0.2,0,0.2,0,0,0.2]),1,1)
fig.add_trace(go.Pie(labels = deliveries_2009['dismissal_kind'].value_counts().index, values =deliveries_2009['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2009",pull = [0,0.2,0,0.2,0,0,0.2]),1,2)
fig.add_trace(go.Pie(labels = deliveries_2010['dismissal_kind'].value_counts().index, values =deliveries_2010['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2010",pull = [0,0.2,0,0.2,0,0,0.2]),1,3)
fig.add_trace(go.Pie(labels = deliveries_2011['dismissal_kind'].value_counts().index, values =deliveries_2011['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2011",pull = [0,0.2,0,0.2,0,0,0.2]),2,1)
fig.add_trace(go.Pie(labels = deliveries_2012['dismissal_kind'].value_counts().index, values =deliveries_2012['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2012",pull = [0,0.2,0,0.2,0,0,0.2]),2,2)
fig.add_trace(go.Pie(labels = deliveries_2013['dismissal_kind'].value_counts().index, values =deliveries_2013['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2013",pull = [0,0.2,0,0.2,0,0,0.2]),2,3)
fig.show()

In [40]:
fig = make_subplots(rows = 12,cols = 1, specs = [[{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],])

fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2008]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2008]['winner'].value_counts().values,
                     name = "2008",orientation = "h"),1,1)
fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2009]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2009]['winner'].value_counts().values,
                     name = "2009",orientation = "h"),3,1)
fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2010]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2010]['winner'].value_counts().values,
                     name = "2010",orientation = "h"),5,1)
fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2011]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2011]['winner'].value_counts().values,
                     name = "2011",orientation = "h"),7,1)
fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2012]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2012]['winner'].value_counts().values,
                     name = "2012",orientation = "h"),9,1)
fig.add_trace(go.Bar(y = matches_first_split[matches_first_split['season'] == 2013]['winner'].value_counts().index, x = matches_first_split[matches_first_split['season'] == 2013]['winner'].value_counts().values,
                     name = "2013",orientation = "h"),11,1)
fig.update_xaxes(showticklabels=False,title = "wins")
fig.update_yaxes(showticklabels=True,title = "teams")
fig.update_layout(title = "Most Team Wins",height = 1200)
fig.show()

In [41]:
fig = make_subplots(rows = 12,cols = 1, specs = [[{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],])

fig.add_trace(go.Bar(y = synergy_gen(deliveries_2008)[0].tolist()[:10], x = synergy_gen(deliveries_2008)[1].tolist()[:10],
                     name = "2008",orientation = "h"),1,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2009)[0].tolist()[:10], x = synergy_gen(deliveries_2009)[1].tolist()[:10],
                     name = "2009",orientation = "h"),3,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2010)[0].tolist()[:10], x = synergy_gen(deliveries_2010)[1].tolist()[:10],
                     name = "2010",orientation = "h"),5,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2011)[0].tolist()[:10], x = synergy_gen(deliveries_2011)[1].tolist()[:10],
                     name = "2011",orientation = "h"),7,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2012)[0].tolist()[:10], x = synergy_gen(deliveries_2012)[1].tolist()[:10],
                     name = "2012",orientation = "h"),9,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2013)[0].tolist()[:10], x = synergy_gen(deliveries_2013)[1].tolist()[:10],
                     name = "2013",orientation = "h"),11,1)
fig.update_xaxes(showticklabels=True)
fig.update_yaxes(showticklabels=True,title = "players")
fig.update_layout(title = "Top 10 Synergies (w.r.t Season)",height = 1300)
fig.show()

## ---------------------------- season wide batch analytics split -2 (2014 - 2019)  ---------------------------

### excluding 2017 ^_^ (it received special test treatment)

In [42]:
matches_second_split = matches[matches['season'].isin(yearlist[6:])]

In [43]:
matches_second_split = matches_second_split[matches_second_split['season'] != 2017].drop(columns = ['index']).reset_index()

In [44]:
matches_second_split = matches_second_split.drop(columns = ['index'])

In [45]:
deliveries_2014 = deliveries[deliveries['match_id'].isin(sorted(matches_second_split[matches_second_split['season'] == 2014]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2015 = deliveries[deliveries['match_id'].isin(sorted(matches_second_split[matches_second_split['season'] == 2015]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2016 = deliveries[deliveries['match_id'].isin(sorted(matches_second_split[matches_second_split['season'] == 2016]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2018 = deliveries[deliveries['match_id'].isin(sorted(matches_second_split[matches_second_split['season'] == 2018]['id'].value_counts().index.tolist()))].reset_index()
deliveries_2019 = deliveries[deliveries['match_id'].isin(sorted(matches_second_split[matches_second_split['season'] == 2019]['id'].value_counts().index.tolist()))].reset_index()

In [46]:
fig = make_subplots(rows =2 ,cols = 3,
                   specs = [[{"type":"pie"},{"type":"pie"},{"type":"pie"}],
                            [{"type":"pie"},None,{"type":"pie"}]])
fig.add_trace(go.Pie(labels = deliveries_2014['dismissal_kind'].value_counts().index, values =deliveries_2014['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2014",pull = [0,0.2,0,0.2,0,0,0.2]),1,1)
fig.add_trace(go.Pie(labels = deliveries_2015['dismissal_kind'].value_counts().index, values =deliveries_2015['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2015",pull = [0,0.2,0,0.2,0,0,0.2]),1,2)
fig.add_trace(go.Pie(labels = deliveries_2016['dismissal_kind'].value_counts().index, values =deliveries_2016['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2016",pull = [0,0.2,0,0.2,0,0,0.2]),1,3)
fig.add_trace(go.Pie(labels = deliveries_2018['dismissal_kind'].value_counts().index, values =deliveries_2018['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2018",pull = [0,0.2,0,0.2,0,0,0.2]),2,1)
fig.add_trace(go.Pie(labels = deliveries_2019['dismissal_kind'].value_counts().index, values =deliveries_2019['dismissal_kind'].value_counts().values,
             title= "top OUT! reasons 2019",pull = [0,0.2,0,0.2,0,0,0.2]),2,3)
fig.show()

In [47]:
fig = make_subplots(rows = 10,cols = 1, specs = [[{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None]])

fig.add_trace(go.Bar(y = matches_second_split[matches_second_split['season'] == 2014]['winner'].value_counts().index, x = matches_second_split[matches_second_split['season'] == 2014]['winner'].value_counts().values,
                     name = "2014",orientation = "h"),1,1)
fig.add_trace(go.Bar(y = matches_second_split[matches_second_split['season'] == 2015]['winner'].value_counts().index, x = matches_second_split[matches_second_split['season'] == 2015]['winner'].value_counts().values,
                     name = "2015",orientation = "h"),3,1)
fig.add_trace(go.Bar(y = matches_second_split[matches_second_split['season'] == 2016]['winner'].value_counts().index, x = matches_second_split[matches_second_split['season'] == 2016]['winner'].value_counts().values,
                     name = "2016",orientation = "h"),5,1)
fig.add_trace(go.Bar(y = matches_second_split[matches_second_split['season'] == 2018]['winner'].value_counts().index, x = matches_second_split[matches_second_split['season'] == 2018]['winner'].value_counts().values,
                     name = "2018",orientation = "h"),7,1)
fig.add_trace(go.Bar(y = matches_second_split[matches_second_split['season'] == 2019]['winner'].value_counts().index, x = matches_second_split[matches_second_split['season'] == 2019]['winner'].value_counts().values,
                     name = "2019",orientation = "h"),9,1)
fig.update_xaxes(showticklabels=False,title = "wins")
fig.update_yaxes(showticklabels=True,title = "teams")
fig.update_layout(title = "Most Team Wins",height = 1200)
fig.show()

In [48]:
fig = make_subplots(rows = 10,cols = 1, specs = [[{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None],
                                              [{'type':"bar",'rowspan':2}],
                                              [None]])

fig.add_trace(go.Bar(y = synergy_gen(deliveries_2014)[0].tolist()[:10], x = synergy_gen(deliveries_2014)[1].tolist()[:10],
                     name = "2014",orientation = "h"),1,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2015)[0].tolist()[:10], x = synergy_gen(deliveries_2015)[1].tolist()[:10],
                     name = "2015",orientation = "h"),3,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2016)[0].tolist()[:10], x = synergy_gen(deliveries_2016)[1].tolist()[:10],
                     name = "2016",orientation = "h"),5,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2018)[0].tolist()[:10], x = synergy_gen(deliveries_2018)[1].tolist()[:10],
                     name = "2018",orientation = "h"),7,1)
fig.add_trace(go.Bar(y = synergy_gen(deliveries_2019)[0].tolist()[:10], x = synergy_gen(deliveries_2019)[1].tolist()[:10],
                     name = "2019",orientation = "h"),9,1)
fig.update_xaxes(showticklabels=True)
fig.update_yaxes(showticklabels=True,title = "players")
fig.update_layout(title = "Top 10 Synergies (w.r.t Season)",height = 1300)
fig.show()